# STT
In diesem Notebook werden zwei verschiedene speach to text Module untersucht. Das erste ist das Google Speech to Text API und das zweite ist das Azure Speech to Text API. Für beide APIs gibt es einen Beispielaufruf. Weiterhin hat es für beide Module eine fertige Implementation, die so benutzt werden kann.
## Google STT
Hierbei handelt es sich um eine Lösung von Google. Diese ist kostenlos. Die Berechnung geschieht in der Cloud.
### API
Nachfolgend sehen wir, wie wir die Google STT API verwenden können.

In [27]:
import speech_recognition as sr

In [28]:
def recognize_speech_from_mic(recognizer, microphone):
    """
    Transkribiert Sprache von Mikrofon und gibt sie aus.

    :param recognizer: (Recognizer) Instanz von sr.Recognizer
    :param microphone: (Microphone) Instanz von sr.Microphone
    :return: (str) Transkribierter Text
    """
    # Überprüfe Typ der `recognizer` und `microphone` Argumente
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` muss `Recognizer` Instanz sein")
    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` muss `Microphone` Instanz sein")

    # Nehme Audio auf
    with microphone as source:
        print("Bitte sprich etwas:")
        recognizer.adjust_for_ambient_noise(source)  # passt sich an Umgebungsgeräusche an
        audio = recognizer.listen(source, timeout=10)  # hört auf das erste Wort oder für 10 Sekunden

    # Versuche, die Sprache zu erkennen
    try:
        print("Analyse des Audios...")
        speech_text = recognizer.recognize_google(audio, language="de-DE")
        print("Du hast gesagt: {}".format(speech_text))
    except sr.UnknownValueError:
        # Wenn die Sprache nicht verstanden wurde
        print("Entschuldigung, ich konnte das nicht verstehen.")
    except sr.RequestError as e:
        # Wenn die API-Anfrage fehlschlägt
        print("Es gab einen Fehler bei der Anfrage; {0}".format(e))
    
    return speech_text if 'speech_text' in locals() else None

In [29]:
# Beispiel-Nutzung
recognizer = sr.Recognizer()
microphone = sr.Microphone()
recognized_text = recognize_speech_from_mic(recognizer, microphone)

Bitte sprich etwas:
Analyse des Audios...
Du hast gesagt: hallo wie geht's


### Klasse GoogleSTT
Hierbei handelt es sich um eine Klasse, welche auf der API basiert. Sie kann so verwendet werden, wie sie ist.

In [30]:
from gpt_bot.stt import GoogleSTT

In [31]:
# Beispiel-Nutzung
google_stt = GoogleSTT(verbose=True)
google_stt.stt()

say something...
you said: hallo wie geht's


"hallo wie geht's"

Fazit zu Google STT
+ Relativ einfach zu verwenden
+ Kostenlos
+ Schnell

- Hat Probleme mit Fremdwörtern und Eigennamen
- Sprache muss definiert werden
- durchschnittliche Performance

## Azure STT
Hierbei handelt es sich um eine Lösung von Microsoft. Diese ist nicht kostenlos. Mit einem Studenten Account bei Azure, kann der Dienst jedoch kostenlos genutzt werden. Die Berechnung geschieht in der Cloud.

In [32]:
subscription_key = "DEIN_AZURE_SUBSCRIPTION_KEY"
region = "DEIN_AZURE_SERVICE_REGION"

### API
Hierbei handelt es sich um ein Beispiel für die Verwendung der Azure cognitive services STT API.

In [33]:
import azure.cognitiveservices.speech as speechsdk

In [34]:
def speech_recognition_azure(subscription_key, region):
    """
    Führt Spracherkennung mit Azure Speech SDK durch und gibt den erkannten Text aus.

    :param subscription_key: (str) Azure Subscription Key.
    :param region: (str) Region, in der der Azure Service erstellt wurde.
    """
    # Konfiguration erstellen
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)

    # Spracherkenner mit der gegebenen Einstellung und Audio-Konfiguration erstellen
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language="de-DE")

    print("Sage etwas...")

    # Einmalige, synchrone Spracherkennung
    result = speech_recognizer.recognize_once()

    # Überprüfen des Ergebnisses
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Erkannt: {}".format(result.text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("Kein Treffer gefunden.")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Spracherkennung abgebrochen: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Fehler-Details: {}".format(cancellation_details.error_details))

In [35]:
# Beispiel-Nutzung
speech_recognition_azure(subscription_key, region)

Sage etwas...
Erkannt: Hallo, wie gehts?


### Klasse AzureSTT
Nachfolgend die Verwendung der fertigen Klasse, welche auf der API basiert.

In [36]:
from gpt_bot.stt import AzureSTT

In [37]:
# Beispiel-Nutzung
azure_stt = AzureSTT(subscription_key, region, verbose=True)
azure_stt.stt()

say something...
you said: Und wie gehts?


'Und wie gehts?'

Fazit zu Azure STT
+ Relativ einfach zu verwenden
+ Gute Performance
+ Gute Erkennung von Fremdwörtern und 
+ Für Studierende kostenlos

- Nicht kostenlos
- Sprache muss definiert werden